# Naive Bayes Classifier - Explainable Machine Learning WS 23/24

<h2> 1. Einführung

### Bayes Classifier Intro

In [ ]:
from IPython.display import Image

png_path = '/Users/fabian/Desktop/datasets/BayesIntro.png'

Image(filename=png_path, width=400, height=300)

### Gliederung

<html>
<ol>
 <li style="font-style: italic;">Einführung</li>
 <li style="font-style: italic;">Preprocessing</li>
 <li style="font-style: italic;">Building the Model</li>
<li style="font-style: italic;">Model Training</li>
<li style="font-style: italic;">Explainability</li>
<li style="font-style: italic;">Predictions and Evaluations</li>
<li style="font-style: italic;">Hyperparametertuning</li>
<li style="font-style: italic;">Principal Component Analysis</li>
</html>

### Poster

https://github.com/eva-f00/bayes_xml/blob/main/Bayes_Poster.pdf

In [ ]:
png_path = '/Users/fabian/Desktop/datasets/Bayes_Poster.jpg'

#Image(filename=png_path)

<h2> 2. Imports, Loading Dataset & Analyze Data 

### Imports

In [ ]:
#in the following code, a Gaussian Naive Bayes Classifier will be build to predict whether a person makes over 50K a year

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv), Python data analysis library
import matplotlib.pyplot as plt # for data visualization purposes
import seaborn as sns # for statistical data visualization; to explore the purpose and target column

# Machine Learning and data analysis

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import fbeta_score

import scipy.stats as stats

### Loading the dataset

In [ ]:
#import dataset as ds

#path = 'C:/Users/evafi/bayes_xml/dataset/adult_income_dataset.csv'
path = '/Users/fabian/Desktop/datasets/adult.csv'
#path = 'C:\\Users\\Natal\\Documents\\Wirtschaftsinformatik_Master\\2.Semester_WS2023-24\\Explainable Machine Learning\\Prüfungsleistung\\adult income dataset.csv'

data = pd.read_csv(path, sep=",")

#top 5 of each column
data.head()

<h2> Analyze the dataset

In [ ]:
# shape dataset
#numer of rows and columns/ features
data.shape

In [ ]:
#data analysis
n_records = data.shape[0]
n_greater_50k = data[data['income'] == '>50K'].shape[0]
n_at_most_50k = data[data['income'] == '<=50K'].shape[0]
greater_percent = (n_greater_50k / n_records) * 100
print("Total numbber of records: {}".format(n_records))
print("Individuals making more than $50.000: {}".format(n_greater_50k))
print("Individuals making at most $50.000: {}".format(n_at_most_50k))
print("Percentage of individuals making more than $50.000: {}%".format(greater_percent))

In [ ]:
#rename column names
col_names = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation', 'relationship',
             'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'income']
data.columns = col_names
data.columns

In [ ]:
# general inforamtion
data.info()

In [ ]:
#Overview of the data
#data.describe(include='all')

<h2>2. Preprocessing the data

<h4> Check values in each variable and replace them / Dealing with missing values

In [ ]:
#check "?" in dataset

col_n = data.columns
num_data = data.shape[0]
for c in col_n:
    num_non = data[c].isin(["?"]).sum()
    if num_non > 0:
        print(c)
        print(num_non)
        print("{0:.2f}%".format(float(num_non) / num_data * 100))

In [ ]:
# replace '?' values in workclass variable with `NaN`
data['workclass'].replace('?', np.NaN, inplace=True)

# replace '?' values in occupation variable with `NaN`
data['occupation'].replace('?', np.NaN, inplace=True)

# replace '?' values in native_country variable with `NaN`
data['native_country'].replace('?', np.NaN, inplace=True)

# impute missing categorical variables with most frequent value

# Fill missing values in 'workclass' column with mode
mode_workclass = data['workclass'].mode()[0]
data['workclass'].fillna(mode_workclass, inplace=True)

# Fill missing values in 'occupation' column with mode
mode_occupation = data['occupation'].mode()[0]
data['occupation'].fillna(mode_occupation, inplace=True)

# Fill missing values in 'native.country' column with mode
mode_native_country = data['native_country'].mode()[0]
data['native_country'].fillna(mode_native_country, inplace=True)

In [ ]:
# Check for missing values again
data.isnull().sum()

In [ ]:
# find categorical variables

categorical = [var for var in data.columns if data[var].dtype=='O']

data[categorical].head()

<h4> Normalization

In [ ]:
# check for cardinality in categorical variables
for var in categorical:
    print(var, ' contains ', len(data[var].unique()), ' labels')

In [ ]:
# get numerical variables
numerical = [var for var in data.columns if data[var].dtype!='O']
print('There are {} numerical variables\n'.format(len(numerical)))
print('The numerical variables are :', numerical)

In [ ]:
# Convert columns with numerical data to numeric data type
numerical_cols = ['age', 'fnlwgt', 'education_num', 'capital_gain', 'capital_loss', 'hours_per_week']
data[numerical_cols] = data[numerical_cols].apply(pd.to_numeric)

In [ ]:
data[numerical_cols]

In [ ]:
corrmat = data[numerical_cols].corr()
print(corrmat)

In [ ]:
plt.figure(figsize=(15,5))
heatmap = sns.heatmap(corrmat, vmin=-1, vmax=1, annot=True)
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':12}, pad=12)

In [ ]:
# Split the data into features and target label
income_raw = data['income']
features_raw = data.drop('income', axis = 1)

In [ ]:
# Visualize the distribution of income
"""sns.countplot(x='income', data=data)
plt.title('Distribution of Income')
plt.xlabel('Income')
plt.ylabel('Count')
plt.show()"""

# Visualize the distribution of age
"""sns.histplot(data['age'], bins=20)
plt.title('Distribution of Age')
plt.xlabel('Age')
plt.ylabel('Count')
plt.show()"""

# Comparison Bayes-EBM: Visualize the income distribution by age
sns.countplot(x='age', hue='income', data=data)
plt.title('Income Distribution by Age')
plt.xlabel('Age')
plt.ylabel('Count')
plt.xticks(rotation=45)
# Set x-axis ticks in steps of 10
plt.xticks(range(0, max(data['age']) + 1, 10))
plt.legend(title='Income', loc='upper right')
plt.show()

In [ ]:
# Comparison Bayes-EBM: Visualize the income distribution by education level
sns.countplot(x='education', hue='income', data=data)
plt.title('Income Distribution by Education Level')
plt.xlabel('Education Level')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.legend(title='Income', loc='upper right')
plt.show()

# Visualize the income distribution by occupation
"""sns.countplot(x='occupation', hue='income', data=data)
plt.title('Income Distribution by Occupation')
plt.xlabel('Occupation')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.legend(title='Income', loc='upper right')
plt.show()"""

In [ ]:
# Visualize the income distribution by race
"""sns.countplot(x='race', hue='income', data=data)
plt.title('Income Distribution by Race')
plt.xlabel('Race')
plt.ylabel('Count')
plt.legend(title='Income', loc='upper right')
plt.show()"""


# Visualize the income distribution by workclass
"""sns.countplot(x='workclass', hue='income', data=data)
plt.title('Income Distribution by Workclass')
plt.xlabel('Workclass')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.legend(title='Income', loc='upper right')
plt.show()"""

In [ ]:
# Log-transform the skewed features
skewed = ['capital_gain', 'capital_loss']
features_log_transformed = pd.DataFrame(data = features_raw)
features_log_transformed[skewed] = features_raw[skewed].apply(lambda x: np.log(x + 1))

In [ ]:
# Visualize the distribution of capital-gain after transformation
"""sns.histplot(features_log_transformed['capital_gain'], bins=20)
plt.title('Distribution of Capital Gain')
plt.xlabel('Capital Gain')
plt.ylabel('Count')
plt.show()

# Visualize the distribution of capital-loss after transformation
sns.histplot(features_log_transformed['capital_loss'], bins=20)
plt.title('Distribution of Capital Loss')
plt.xlabel('Capital Loss')
plt.ylabel('Count')
plt.show()"""

In [ ]:
from sklearn.preprocessing import MinMaxScaler
# Initialize a scaler, then apply it to the features
scaler = MinMaxScaler()
numerical = ['age', 'education_num', 'capital_gain', 'capital_loss', 'hours_per_week']

features_log_minmax_transform = pd.DataFrame(data)
features_log_minmax_transform[numerical] = scaler.fit_transform(features_log_minmax_transform[numerical])
# Show an example of a record with scaling applied
display(features_log_minmax_transform.head(n=5))

In [ ]:
#preprocessing categorial features

features_log_minmax_transform.head(1)

In [ ]:
# Transform Categorial into Numerical
# One-hot encode the 'features_log_minmax_transform' data 
features_final = pd.get_dummies(features_log_minmax_transform)

# Encode the 'income_raw' data to numerical values
income = income_raw.map({'<=50K':0,'>50K':1})

# Print the number of features after one-hot encoding
encoded = list(features_final.columns)
print("{} total features after one-hot encoding.".format(len(encoded)))

# See the encoded feature names
print (encoded)

<h2>3. Building the Model

<h4>Declare feature vector and target variable

In [ ]:
X = data.drop(['income'], axis=1)

y = data['income']
y = y.map({'<=50K':0, '>50K':1})

<h4> Split data into separate training and test set 

In [ ]:
# split X and y into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [ ]:
# check the shape of X_train and X_test
X_train.shape, X_test.shape

<h4> Feature Engineering

In [ ]:
# check data types in X_train
X_train.dtypes

In [ ]:
# display categorical variables
categorical = [col for col in X_train.columns if X_train[col].dtypes == 'O']
categorical

In [ ]:
# display numerical variables
numerical = [col for col in X_train.columns if X_train[col].dtypes != 'O']
numerical

<h4> Encode categorical variables

In [ ]:
# import category encoders
import category_encoders as ce

# encode remaining variables with one-hot encoding
encoder = ce.OneHotEncoder(cols=['workclass', 'education', 'marital_status', 'occupation', 'relationship', 
                                 'race', 'sex', 'native_country'])

X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

In [ ]:
#print
X_train.head()

In [ ]:
X_train.shape, X_test.shape

In [ ]:
# Comparison Bayes-EBM:
#marginal plot for provided data
#how is the training data distributed over a specific range of value
#Pearson correlation coefficient to interpret the linear relationship of these features with respect to the target
graph = sns.jointplot(data= data, x=X_train['age'], y=y_train, alpha = 0.1)
r, p = stats.pearsonr(X_train['age'], y_train)
phantom, = graph.ax_joint.plot([], [], linestyle="", alpha=0)
graph.ax_joint.legend([phantom],['r={:f}, p={:f}'.format(r,p)])
plt.show()

<h4> Feature Scaling

In [ ]:
cols = X_train.columns

from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train = pd.DataFrame(X_train, columns=[cols])
X_test = pd.DataFrame(X_test, columns=[cols])
X_train.head()

<h2>4. Model training

In [ ]:
# train a Gaussian Naive Bayes classifier on the training set
# instantiate the model
gnb = GaussianNB()

# fit the model
gnb.fit(X_train, y_train)

<h2> 5. Explainability

<h4> Directed Acyclic Graph

In [ ]:
"""import networkx as nx

# Get mean for each feature and class
means = gnb.theta_

# Get feature names
feature_names = X.columns

# Create a directed graph using networkx
G = nx.DiGraph()

# Add nodes for features
for feature in feature_names:
    G.add_node(feature)

# Add edges based on conditional dependencies
for i, feature in enumerate(feature_names):
    for j, class_label in enumerate(['0', '1']):
        mean_label = f'Mean_{class_label}'
        G.add_node(mean_label)
        G.add_edge(mean_label, feature, label=f'Mean: {means[j, i]:.2f}')

# Visualize the graph
pos = nx.spring_layout(G)
labels = nx.get_edge_attributes(G, 'label')
nx.draw(G, pos, with_labels=True, font_weight='bold', node_size=2000, node_color='lightblue', font_size=8)
nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)
plt.show()"""


<h4> Directed Acyclic Graph with weighted mean score

In [ ]:
"""import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

# Get mean for each feature and class
means = gnb.theta_

# Get feature names
feature_names = X.columns

# Create a directed graph using networkx
G = nx.DiGraph()

# Add nodes for features
for feature in feature_names:
    G.add_node(feature)

# Dictionary to store the weighted mean absolute score for each feature
weighted_mean_absolute_scores = {}

# Add edges based on conditional dependencies and calculate weighted mean absolute score for each feature
for i, feature in enumerate(feature_names):
    weighted_mean_absolute_score = 0.0
    for j, class_label in enumerate(['0', '1']):
        mean_label = f'Mean_{class_label}'
        G.add_node(mean_label)
        weight = means[j, i]  # Use the mean as the weight
        weighted_mean_absolute_score += np.abs(weight)  # Accumulate absolute weights
        G.add_edge(mean_label, feature, label=f'Mean: {means[j, i]:.4f}')

         # Reverse the direction of the arrow by swapping the order of nodes
        #G.add_edge(feature, mean_label, label=f'Mean: {means[j, i]:.4f}')

    # Store the weighted mean absolute score for the current feature
    weighted_mean_absolute_scores[feature] = weighted_mean_absolute_score

# Display the weighted mean absolute score for each feature in descending order
sorted_scores = sorted(weighted_mean_absolute_scores.items(), key=lambda x: x[1], reverse=True)
for feature, score in sorted_scores:
    print(f"Feature: {feature}, Weighted Mean Absolute Score: {score:.4f}")

# Visualize the graph
pos = nx.spring_layout(G)
labels = nx.get_edge_attributes(G, 'label')
nx.draw(G, pos, with_labels=True, font_weight='bold', node_size=2000, node_color='lightblue', font_size=8)
nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)
plt.show()

# Comparison Bayes-EBM:"""

<h4> Directed Acyclic Graph: Dependencies betweeen Features

In [ ]:
"""import bnlearn as bn
import matplotlib.pyplot as plt

# Assuming you have your data and trained DAG
DAG = bn.structure_learning.fit(data)
model_mle = bn.parameter_learning.fit(DAG, data, methodtype='maximumlikelihood')

# Plotting the DAG with modified axis labels and title
plt.figure(figsize=(8, 6))  # Define the figure size

# Plot the DAG
bn.plot(model_mle)

# Retrieve the current Axes object
ax = plt.gca()

ax.set_title('bnlearn Directed Acyclic Graph (DAG)')  # Set the title
ax.set_xlabel('Modified X-axis label')  # Set the modified X-axis label
ax.set_ylabel('Modified Y-axis label')  # Set the modified Y-axis label

plt.show()"""

<h4> Bedingte Wahrscheinlichkeiten

In [ ]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# Select the relevant features for the specific scenario
specific_data = pd.DataFrame({'education': ['Bachelors'], 'age': [20]})

# Convert column names to strings
specific_data.columns = specific_data.columns.astype(str)

# Combine specific data with the original dataset to ensure all categories are present
combined_data = pd.concat([data[['education', 'age']], specific_data])

# Use OneHotEncoder for categorical variables
encoder = OneHotEncoder(sparse=False) 
encoder.fit(combined_data)

# Transform the specific data into a DataFrame
specific_data_encoded = encoder.transform(specific_data)
column_names = encoder.get_feature_names_out(specific_data.columns)
specific_data_encoded_df = pd.DataFrame(specific_data_encoded, columns=column_names)


if specific_data_encoded_df.shape[1] < 105:
    missing_columns = 105 - specific_data_encoded_df.shape[1]
    for i in range(missing_columns):
        specific_data_encoded_df[f'extra_feature_{i}'] = 0

# Calculate probabilities for the classes 
probabilities = gnb.predict_proba(specific_data_encoded_df)
print("Class probabilities as percentages:")
probabilities_percent = probabilities * 100
for class_idx, prob in enumerate(probabilities_percent[0]):
    print(f"Class {class_idx}: {prob:.2f}%")

In [ ]:
"""import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import networkx as nx
import numpy as np

# Your existing code for creating the graph 'G'

# Assign arbitrary z-coordinates for 3D visualization
z_coords = {node: 0 for node in G.nodes()}  # Assigning a default value for z-coordinate (you may need to adjust this)

# Plotting in 3D
fig = plt.figure(figsize=(12, 10))  # Increase figure size
ax = fig.add_subplot(111, projection='3d')

# Get x, y, z coordinates for nodes
nodes = G.nodes()
feature_nodes = [node for node in nodes if isinstance(node, str) and 'Mean' not in node]

# Create a colormap to generate distinct colors for each feature
colormap = plt.cm.get_cmap('tab20', len(feature_nodes))

# Plot nodes with different colors for each feature
for idx, node in enumerate(feature_nodes):
    color = colormap(idx)
    ax.scatter(pos[node][0], pos[node][1], z_coords[node], color=color, s=800, label=node)  # Increase marker size

# Plot edges
for edge in G.edges():
    ax.plot([pos[edge[0]][0], pos[edge[1]][0]], [pos[edge[0]][1], pos[edge[1]][1]], [z_coords[edge[0]], z_coords[edge[1]]], 'k-')

# Add labels
for node in nodes:
    ax.text(pos[node][0], pos[node][1], z_coords[node], node, color='black', fontsize=10)  # Increase label font size

ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.set_zlabel('Z-axis')

plt.legend()
plt.show()"""

In [ ]:
"""# Assuming you have 'age' and 'sex' as features
new_data = pd.DataFrame({'age': [25], 'sex': ['Female']})  # Adjust the values accordingly

# Encoding categorical variables like 'sex' to numeric values
new_data['sex'] = new_data['sex'].map({'Male': 0, 'Female': 1})  # Assuming binary encoding

# If 'gnb' expects 105 features, create dummy features for the rest
all_features = pd.DataFrame()

for i in range(3, 106):
    all_features[f'feature_{i}'] = [0]  # Initialize all other features with 0

# Update 'age' and 'sex' in all_features with the provided values
all_features['age'] = new_data['age']
all_features['sex'] = new_data['sex']

# Predict the probabilities for the new_data instance
predicted_probabilities = gnb.predict_proba(all_features)

# Assuming your classes or labels are ['<=50K', '>50K']
classes = gnb.classes_

# Print the probabilities for each income category
for idx, label in enumerate(classes):
    print(f"Probability of {label} income: {predicted_probabilities[0][idx]}")"""


<h4>Feature Importance

In [ ]:
# Get mean for each feature and class
means = gnb.theta_

# Get feature names
feature_names = X.columns

# Plot the mean values
plt.figure(figsize=(10, 6))
for i, feature in enumerate(feature_names):
    plt.barh([f'{feature}_class_0', f'{feature}_class_1'], means[:, i], label=feature)

plt.xlabel('Mean Value')
plt.title('Feature Importance (Mean Values)')
plt.legend()
plt.show()

In [ ]:
import networkx as nx
# Get mean for each feature and class
means = gnb.theta_

# Get feature names
feature_names = X.columns

# Create a directed graph using networkx
G = nx.DiGraph()

# Dictionary to store the weighted mean absolute score for each feature
weighted_mean_absolute_scores = {}

# Add edges based on conditional dependencies and calculate weighted mean absolute score for each feature
for i, feature in enumerate(feature_names):
    weighted_mean_absolute_score = 0.0
    for j, class_label in enumerate(['0', '1']):
        mean_label = f'Mean_{class_label}'
        G.add_node(mean_label)
        weight = means[j, i]  # Use the mean as the weight
        weighted_mean_absolute_score += np.abs(weight)  # Accumulate absolute weights
        G.add_edge(mean_label, feature, label=f'Mean: {means[j, i]:.4f}')

    # Store the weighted mean absolute score for the current feature
    weighted_mean_absolute_scores[feature] = weighted_mean_absolute_score

# Display the weighted mean absolute score for each feature in a horizontal bar diagram
sorted_scores = sorted(weighted_mean_absolute_scores.items(), key=lambda x: x[1], reverse=True)
features, scores = zip(*sorted_scores)

plt.figure(figsize=(10, 6))
plt.barh(features, scores, color='lightblue', edgecolor='black', alpha=0.8)
plt.xlabel('Weighted Mean Absolute Score')
plt.title('Feature Importance (Weighted Mean Absolute Scores)')
plt.show()

# Comparison Bayes-EBM:

In [ ]:
# Get mean for each feature and class
means = gnb.theta_

# Get feature names
feature_names = X.columns

# Plot the mean values
plt.figure(figsize=(10, 6))
for i, feature in enumerate(feature_names):
    plt.barh(feature, means[:, i])

plt.xlabel('Mean Value')
plt.title('Feature Importance (Mean Values)')
plt.legend()
plt.show()

<h4>Partial Dependence Plots (PDPs)

In [ ]:
feature_name = 'age'

# Create the PDP
unique_values = np.unique(X_test[feature_name])
pdp_values = []

for value in unique_values:
    X_pdp = X_test.copy()
    X_pdp[feature_name] = value
    pdp_values.append(gnb.predict_proba(X_pdp)[:, 1].mean())

# Plot the PDP
plt.plot(unique_values, pdp_values, marker='o')
plt.xlabel(feature_name)
plt.ylabel('Average Predicted Probability')
plt.title(f'Partial Dependence Plot for {feature_name}')
plt.show()

<h2>6. Predictions and Evaluations

In [ ]:
# Performance Evaluation
# Counting the ones as this is the naive case. Note that 'income' is the 'income_raw' data encoded to numerical values done in the data preprocessing step.
TP = np.sum(income) 
# Specific to the naive case
FP = income.count() - TP
# No predicted negatives in the naive case
TN = 0 
FN = 0 

# Calculate accuracy, precision and recall
accuracy = TP / (TP + FP + TN + FN)
recall = TP / (TP + FN)
precision = TP / (TP + FP)

# Calculate F-score using the formula above for beta = 0.5 and correct values for precision and recall.
beta = 0.5
fscore = (1 + beta**2) * ((precision * recall) / ((beta**2) * precision + recall))

# Print the results 
print("Naive Predictor: [Accuracy score: {:.4f}, F-score: {:.4f}]".format(accuracy, fscore))

<h4> Predict the results

In [ ]:
y_pred = gnb.predict(X_test)
y_pred

<h4> Check accuracy score

In [ ]:
print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

In [ ]:
# compare the train-set and test-set accuracy
y_pred_train = gnb.predict(X_train)
y_pred_train
print('Training-set accuracy score: {0:0.4f}'. format(accuracy_score(y_train, y_pred_train)))

In [ ]:
# Check for overfitting and underfitting
# print the scores on training and test set

print('Training set score: {:.4f}'.format(gnb.score(X_train, y_train)))

print('Test set score: {:.4f}'.format(gnb.score(X_test, y_test)))

In [ ]:
# Compare model accuracy with null accuracy
# check class distribution in test set
y_test.value_counts()

In [ ]:
# Make predictions using the model
predictions = (gnb.fit(X_train, y_train)).predict(X_test)

# Report accuracy and fscore
print("Accuracy score on testing data {:.4f}".format(accuracy_score(y_test, predictions)))
print("F-score on testing data: {:.4f}".format(fbeta_score(y_test, predictions, beta = 0.5)))

<h4> Confusion matrix

In [ ]:
# Print the Confusion Matrix and slice it into four pieces
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print('Confusion matrix\n\n', cm)
print('\nTrue Positives(TP) = ', cm[0,0])
print('\nTrue Negatives(TN) = ', cm[1,1])
print('\nFalse Positives(FP) = ', cm[0,1])
print('\nFalse Negatives(FN) = ', cm[1,0])

In [ ]:
# visualize confusion matrix with seaborn heatmap
cm_matrix = pd.DataFrame(data=cm, columns=['Actual Positive:1', 'Actual Negative:0'], 
                                 index=['Predict Positive:1', 'Predict Negative:0'])
sns.heatmap(cm_matrix, annot=True, fmt='d', cmap='YlGnBu')

<h4> Classification metrices

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

In [ ]:
TP = cm[0,0]
TN = cm[1,1]
FP = cm[0,1]
FN = cm[1,0]

# print classification accuracy
classification_accuracy = (TP + TN) / float(TP + TN + FP + FN)
print('Classification accuracy : {0:0.4f}'.format(classification_accuracy))

In [ ]:
# print classification error
classification_error = (FP + FN) / float(TP + TN + FP + FN)
print('Classification error : {0:0.4f}'.format(classification_error))

<h4> Class probabilities

In [ ]:
y_pred_prob = gnb.predict_proba(X_test)[0:10]
# store the probabilities in dataframe
y_pred_prob_df = pd.DataFrame(data=y_pred_prob, columns=['Prob of - <=50K', 'Prob of - >50K'])

# store the predicted probabilities for class 1 - Probability of >50K
y_pred1 = gnb.predict_proba(X_test)[:, 1]

In [ ]:
# plot histogram of predicted probabilities

# adjust the font size 
plt.rcParams['font.size'] = 12

# plot histogram with 10 bins
plt.hist(y_pred1, bins = 10)

# set the title of predicted probabilities
plt.title('Histogram of predicted probabilities of salaries >50K')

# set the x-axis limit
plt.xlim(0,1)

# set the title
plt.xlabel('Predicted probabilities of salaries >50K')
plt.ylabel('Frequency')

In [ ]:
# Combine X_train and y_train into a single DataFrame
train_data = pd.concat([X_train, pd.Series(y_train, name='Target')], axis=1)
# Calculate standard deviations for each feature and class
std_devs = train_data.groupby('Target').std()
# Print the standard deviations
print("Standard Deviations:")
print(std_devs)

<h2> 7. Hyperparametertuning

In [ ]:
from sklearn.model_selection import GridSearchCV, cross_val_score

In [ ]:
np.logspace(0,-9, num=10)

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold

cv_method = RepeatedStratifiedKFold(n_splits=5, 
                                    n_repeats=3, 
                                    random_state=999)

In [ ]:
params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}

gs_NB = GridSearchCV(estimator=gnb, 
                     param_grid=params_NB, 
                     cv=cv_method,
                     verbose=1, 
                     scoring='accuracy')


gs_NB.fit(X_train, y_train);

In [ ]:
gs_NB.best_params_

In [ ]:
gs_NB.best_score_

In [ ]:
results_NB = pd.DataFrame(gs_NB.cv_results_['params'])
results_NB['test_score'] = gs_NB.cv_results_['mean_test_score']

In [ ]:
plt.plot(results_NB['var_smoothing'], results_NB['test_score'], marker = '.')    
plt.xlabel('Var. Smoothing')
plt.ylabel("Mean CV Score")
plt.title("NB Performance Comparison")
plt.show()

<h2> 8. Principal Component Analysis

In [ ]:
from sklearn.decomposition import PCA
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import fbeta_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

In [ ]:
result = []
param_distribution = {'var_smoothing': np.logspace(0, -9, num=100)}
scoring = 'accuracy'  

for i in range(1, 13):
    # PCA durchführen
    pca = PCA(n_components=i)
    X_train_pca = pca.fit_transform(X_train)
    
    # RandomizedSearchCV für den Naive Bayes Classifier
    search_cv = RandomizedSearchCV(GaussianNB(), param_distribution, scoring=scoring, n_jobs=-1,
                                   cv=StratifiedKFold(n_splits=10, shuffle=True), refit=scoring)
    search_cv.fit(X_train_pca, y_train)
    best_model = search_cv.best_estimator_

    # Testdaten transformieren
    X_test_pca = pca.transform(X_test)
    
    # Vorhersagen
    y_pred = best_model.predict(X_test_pca)
    
    # Modellbewertung
    f1 = fbeta_score(y_test, y_pred, beta=1, pos_label=1)
    acc = accuracy_score(y_test, y_pred)
    
    print(f"{i} {acc} {f1}")
    
    # Ergebnisse speichern
    result.append((i, acc, f1, pca, best_model))

In [ ]:
# Finde den Index der Zeile mit der höchsten Genauigkeit
best_index = np.argmax(np.array(result)[:, 1])

# Extrahiere die Informationen für die beste Zeile
best_row = result[best_index]

# Extrahiere die Anzahl der Hauptkomponenten und die PCA-Objekt
best_components_count = best_row[0]
best_pca = best_row[3]

# Extrahiere die Namen der drei Hauptkomponenten mit den höchsten Ladevektoren (Betrag)
top_indices = np.argsort(np.abs(best_pca.components_))[:, -3:]
top_feature_names = X_train.columns[top_indices.flatten()]

# Gib die Informationen aus
print("Beste Zeile mit höchster Genauigkeit:")
print(f"Anzahl der Hauptkomponenten: {best_components_count}")
print(f"Namen der Hauptkomponenten mit höchstem Betrag der Ladevektoren:")
print(top_feature_names)
print(f"Genauigkeit: {best_row[1]}")
print(f"F1-Score: {best_row[2]}")

In [ ]:
# Extrahiere die eindeutigen Namen der Merkmale
unique_feature_names = set([name[0] for name in top_feature_names])

# Gib die eindeutigen Namen der Merkmale aus
print("Eindeutige Namen der Hauptkomponenten:")
print(unique_feature_names)

In [ ]:
# Loadings Matrix für die ersten drei Hauptkomponenten
loadings_matrix = pca.components_

# Drucke die Loadings Matrix
print("Loadings Matrix:")
print(loadings_matrix)

In [ ]:
# Plot für jede Hauptkomponente
for i in range(loadings_matrix.shape[0]):
    plt.figure(figsize=(8, 2))
    plt.bar(range(len(loadings_matrix[i])), loadings_matrix[i])
    plt.title(f'Loadings for PC{i+1}')
    plt.xlabel('Feature Name')
    plt.ylabel('Loading Value')
    plt.show()